# Homework 4: Applied ML

We will work with a by-now classic dataset from Robert LaLonde's study "Evaluating the Econometric Evaluations of Training Programs" (1986). The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program. Your task is to determine the effectiveness of the "treatment" represented by the job training program.

We import all the packages needed here and define plotting functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def densplot(columns):
    fig, ax = plt.subplots()
    for i,v in enumerate(columns):
        sns.distplot(v, ax=ax, kde_kws={"label": i})
    plt.show()

We start by importing the data.

#### Dataset description

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)

In [ ]:
data = pd.read_csv('lalonde.csv', index_col=0)
data.head()

### 1. A naive analysis

In this part, we compare the distribution of the outcome variable (`re78`) between the two groups (control and test)

In [ ]:
grouped = data.groupby('treat')
grouped0 = grouped.get_group(0)
grouped1 = grouped.get_group(1)

In [ ]:
densplot([grouped0['re78'],grouped1['re78']])

**What might a naive "researcher" conclude from this superficial analysis?**
A naive observer would think that the two distributions are pretty similar even though there seems to be more people that didn't follow the program with a high income.


In [ ]:
sns.boxplot(data=[grouped0['re78'],grouped1['re78']])
plt.show()